## Uncovering Social Media Trends With ETPOL
### COSC-247 Machine Learning: Application Final Project Writeup
### Fynn Hayton-Ruffner

### Overview

Identifying political bias has always been a topic of great fascination for machine learning practicioners. Though this topic spans a wide breadth of distinct application areas, particular attention is paid to bias in news articles (CITE). This is not without good reason; identifying bias is a skill essential to any education that values critical thinking. While the value of article classififcation is undeniable, a gap in the modern literature exists for working with the political ideology of social media posts. In the past, many simpler, though remarkably effective methods were applied to identify the ideology of twitter posts (CITE). These studies used Decision Trees and Support Vector Machines, which, while very effective tools, can only make sense of sequential textual data as a bag of words. THIS STUDY, worked with youtube titles...While this study used transformeres. Though this is at its core an application project, I was interested to see how well the transformer model I created (which I'm calling ETPOL), measured up against these modern benchmarks. I was not expecting to outperform Google's BERT model due to an incomprehensible gap in skill, resources, and time, but I was looking for something in the range of 82 - 90%. The bottom of this range just exceeds the most stellar performances from the standard machine learning models that I could find in less recent, prior research. I was hopeful that ETPOL could beat those models due to the transformer's exceptional ability to handle sequential data, but admittedly a little worried that a dataset on the smaller end of things (~150,000) might hold it back. Nonetheless, 82-90% was my designated range of sucess, which I managed to achieve. I then applied the resulting model to the TWITTER THREADS....In this writeup, I begin by describing the data and ETPOL, after which I detail the entire training process and resulting model, followed by its application to the twitter threads of interest, and finishing with a discussion of conclusions, implications, and future work.

##### The Data
To start, I had to find a sizeable number of social media posts labeled by political inclination. Luckily, I found two datasets on kaggle that had just the thing: [Messy Twitter Dataset](https://www.kaggle.com/datasets/cascathegreat/politicaltweets) with ~130,000 posts, and [Clean Reddit Dataset](https://www.kaggle.com/datasets/neelgajare/liberals-vs-conservatives-on-reddit-13000-posts/data) with 13,000 posts. However, while I was lucky to find these datasets, they were structured completely differently, so combining them into one took a lot of cleaning and reformatting. I have the full cleaning process in 'clean_data.ipynb' but to sum up, I reformatted the post content of the messy dataset (which was such a pain), removed everything that wasn't alphanumeric or punctuation with regex, renamed columns for consistency, and dealt with invisible unicode chars. When I hopefully use the resulting model on twitter threads, I will have to do apply this same cleaning process to that data for consistency. Nonetheless, after all that, I was able to join the two datasets together to create an 'all_posts.csv' file with 146,478 rows and two columns: 'content' & 'affiliation' (with values left or right); this size is far greater than anything I worked with this semester in this class but it is still relatively small in the grand scheme of ML. For reference, both the polished dataset and the raw csv files are in the 'data' directory of this project.

In [1]:
import pandas as pd
from pathlib import Path

df=pd.read_csv(Path('data/all_posts.csv'))
df.head()

,content,affiliation
0,when you look at the history of big social mov...,left
1,it was great to be back in new jersey! there's...,left
2,"virginians delivered for me twice, and now im ...",left
3,some of the most important changes often start...,left
4,glad i had a chance to talk with our new champ...,left


##### The Tokenizer
After preprocessing the data, the next step in text analysis is to create a tokenizer that maps text to the numerical format necessary to pass it through a transformer. I document the process in 'tokenizer.py' but will summarize it here. To start, I created a word piece tokenizer following the tutorial on [huggingface.com](https://huggingface.co/learn/llm-course/en/chapter6/8). Word piece is a subword tokenizer that breaks words into meaningful sub-units. For example, 'modernization' may become 'modern' & 'ization' since 'ization' may be encountered as a common suffix in the vocabulary. This tackles the common issue of unknown words in new data. Since the tokenizer always starts from every character encountered in the vocab, as long as the input text doesn't have any new characters it will be able to handle it. Given the size of the dataset and that I formatted the input data to only include alphanumeric chars and punctuation, it is very unlikely for the tokenizer to encounter characters not already in its vocabularly. After training the tokenizer on my dataset in tokenizer.py, I its state in the 'wp_tokenizer' directory. It has a 20,000 token vocabulary, which was more than enough to account at least for all the unique characters in my dataset. I provide an example below of how the tokenizer encodes a string into its numerical representation and back again. You'll notice if you run it that not only is the decoded text string normalized (to lower case and uniform whitespace separation), but a new [CLS] token appears at the front. It is standard practice in text classification tasks to include this special CLS token as the token from which the class probabilities of the entire text are extracted (CITE BERT).

In [3]:
from transformers import PreTrainedTokenizerFast
from pathlib import Path 

test_str = "I am a test string!"
tokenizer = PreTrainedTokenizerFast.from_pretrained(Path('wp_tokenizer'), model_max_length=25)

tokenized = tokenizer.encode(test_str)
print(f'Tokens: {tokenized}')
print(f'Decoded tokens: {tokenizer.decode(tokenized)}')

Tokens: [2, 33, 157, 25, 1530, 19216, 3]
Decoded tokens: [CLS] i am a test string !


##### ETPOL
As mentioned above, I decided to try to code my own encoder model using the pytorch module, since transformers tend to outperform other machine learning model on text related tasks. The reason for this gap is attention, the key process of a transformer that enriches the meaning of every token by the context that surrounds it (kind of like how we interpret text). To create the model, I followed pure encoder structure laid out by [Attention is All You Need](https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf). Below I detail the flow of any input sequence of text through the model (writing this out always helps me better understand what this does). I set the max context length of ETPOL to a reasonable 512 tokens, so to make things simpler, anything longer gets truncated and anything shorter gets padded.

Input X shape: (512) -> 512 tokens in input sequence

1: Embedding layer (20,000 x d_model matrix): holds the embedding vector (of size d_model) of every token in the vocabulary (20,000 total). The embedding is meant to encode the meaning of every token numerically. It is the values within each embedding vector that are updated by the encoder with the idea that, by the end of the process, the output embedding of each token comes to represent the reality of what that token means within the context of the sequence. Each input token passes through this layer by indexing its corresponding row in the matrix, and thus every token is transformed into its corresponding embedding vector.

X shape: (512 x d_model): now every token is represented by an embedding vector of dimension d_model

2: Positional Encoder: The positional encoder is a 512 x d_model matrix that, in each row, holds an embedding for every possible position in the input sequence. For example, the token in the 5th position in input sequence has the embedding of the 5th row of this matrix added to it. This process happens for every token embedding in the input

X shape: (512 x d_model): the same shape, but each token embedding has been encoded with positional information

3: Encoder block:
First step: Attention:
    Attention begins by transforming any input token into 3 distinct embeddings via weight matrices: 
        1) the query: the vector represenation of an input embedding as the current token being processed in the sequence
        2) the key: the vector representation of an input embedding as a preceding token in the sequence
        3) the value: the actual value representaiton of a preceding token - used to update the current token after being scaled by its similarity (see below)
    For every input token i, the following process takes place for all preceding tokens j:
        The 'similarity' between i as a query vector and j as a key vector is computed by scaled dot product (which divides by the shape of the key vector). This similarity score is normalized with softmax and serves as scaler for value vector of j. The intuition is that the more similar the two tokens are, the more influence the j value vector will have on the encoded ouput of i. Thus, every preceding token is weighted by its similarity to the current token, and these products are summed up to produce an output vector which is then added back to the initial input embedding of token i. Theoretically, i now contains a more richer reprsentation of that tokens meaning by attending to its context.
    MULTI-HEAD ATTENTION: In practice, the attention process described above is divided among multiple heads, each of which handle different sections of any input embedding. The idea here is that each head learns to be responsible for different linguistic patterns between the token and the context. The query, key, and value vectors for every token in each head are shape d_model/num_heads, and the resulting attention output from each head for any token i is concatenated together prior to updating the input embedding of i.

X shape: (512 x d_model): the same shape, but each token embedding has been updated significantly by the attention output
Next: FeedForward MLP
    After attention, the other core component of an encoder block is the feedforward neural net. The input layer is transformed into a higher dimension (d_hidden), flows through the hidden layers that apply the ReLU activation function to weighted sum of each node (at least that's what I did), whose ouput is transposed back to the same shape as the input. Neural networks excel at discovering non linear relationships between inputs; the encoder takes advantage of that here. The output of the neural net is again shape 512 x d_model, holding new embedding representations from each token. These values are added to the attention updated embeddings.

* I'm leaving out layer normalization and dropout, but note that I did apply those methods in each encoder block
The input passes through an arbitrary number of these encoder blocks, the idea being that by the end of the stack, their embeddings hold rich contextual and linguistic information that will result in better performance on the task at hand

X shape: (512 x d_model): the same shape, but each token embedding has been updated significantly by the attention output, and feedforward.


4: Linear output: after being passed through the linear output, I extract the [CLS] token, which is inserted at the beginning of every input sequence. This token is meant to contain the label for classfication tasks after sufficient training, and is a common method for classfication in transformers. The embedding of the token is reduced from shape d_model to shape 2 (for two classes) by a linear layer. The two values here are meant to hold the class probabilites of the input sentence.

X shape (2): just raw logit probs for each class

Below is an example pass through the model using a random observation in the dataset.

In [ ]:
import torch
import torch.nn as nn
from transformers import PreTrainedTokenizerFast
from models import ETPOL
from pathlib import Path
from PostsDataset import PostsDataset

context_length = 512
d_model = 512

model = ETPOL(
    vocab_size=20000,
    context_length=context_length,
    d_model=d_model,
    num_heads=8,
    num_hidden_layers=2,
    d_hidden=2048,
    num_encoders=8
)

tokenizer = PreTrainedTokenizerFast.from_pretrained(Path('wp_tokenizer'), model_max_length=context_length)
# creates a dataset object from the dataframe, which internally handles tokenizing the inputs
dataset = PostsDataset(df, tokenizer, context_length)
rand_example = dataset.__getitem__(3)


: 

In [ ]:
reverse_label_map = {1: 'left', 0: 'right'} # this is the scheme I used to encode labels
inputs = rand_example['input_ids'].unsqueeze(0) # add the batch dimension (expected by model, used to speed up training)
label = rand_example['labels']
outputs = model(inputs)
print(outputs)
print(outputs.shape) # class probs, one for each class, first dimension is batch size

In [ ]:
# lets look at the result! this is an untrained model so there is no significance to its prediction, especially given the binary nature of the classification
print(f"input post was: {tokenizer.decode(rand_example['input_ids'])}")
print(f"its label was: {reverse_label_map[int(label)]}")
print(f"It was predicted to be: {reverse_label_map[int(torch.argmax(outputs, dim=-1))]}")

##### TRAINING & HYPERPARAMETER TUNING